$$
\DeclareMathOperator*{\argmin}{argmin}
\DeclareMathOperator*{\E}{\mathbb E}
\newcommand{\R}{\mathbb R}
$$

<!-- SOLUTION CELL -->
<h2 style="color: red">This is the solutions file!</h2>

<span style="color: red">We strongly recommend not looking at this file, which contains the solutions to all the exercises, until after the practical is over. Use [`practical.ipynb`](practical.ipynb) instead.</span>

In [ ]:
# SOLUTION CELL
# This is put straight into practical.ipynb so it renders untrusted...
from IPython.display import display, Markdown
with open('README-setup.md') as f:
    display(Markdown(f.read()))

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='notebook')

import functools

import numpy as np
import sklearn
from sklearn import model_selection
import torch

from ds3_support import as_tensors, LazyKernel

# download some datasets
from torchvision.datasets import MNIST, Omniglot
MNIST(root='data', download=True)
Omniglot(root='data', download=True)

# Feel free to read ahead while the datasets are downloading.

## Ridge regression
Okay, time to get going.

We're going to start with implementing kernel ridge regression. First, here's a toy dataset to test your code on:

In [ ]:
# SOLUTION CELL
from sklearn.metrics.pairwise import rbf_kernel

def make_toy(n_pts, dist_seed=17, samp_seed=12):
    drs = np.random.RandomState(seed=dist_seed)
    inducing = drs.uniform(0, 1, size=50)
    w = drs.normal(size=inducing.shape[0])
    gamma = 1 / (2 * .05**2)
    print(f"||f||_H^2: {w @ rbf_kernel(inducing[:, None], gamma=gamma) @ w :.3}")
    
    srs = np.random.RandomState(seed=samp_seed)
    X = srs.uniform(0, 1, size=n_pts)
    ymean = rbf_kernel(X[:, None], inducing[:, None], gamma=gamma) @ w
    y = ymean + srs.normal(scale=.5, size=ymean.shape)
    return X[:, None].astype(np.float32), y.astype(np.float32)

X, y = make_toy(500)
np.savez('data/ridge-toy.npz', X=X, y=y)

In [ ]:
with np.load('data/ridge-toy.npz') as f:
    toy_X = f['X']
    toy_y = f['y']
toy_X_train, toy_X_test, toy_y_train, toy_y_test = model_selection.train_test_split(
    toy_X, toy_y, train_size=200)

fig, ax = plt.subplots()
ax.scatter(toy_X_train[:, 0], toy_y_train)

This data looks like it could be well-modeled by a Gaussian RBF kernel. We'll need a fairly small bandwidth. See the minimum a little above 0.15 and the maximum a little less than 0.25? The Gaussian kernel still has a fair amount of influnce one bandwidth away, so we want to make the bandwidth a little smaller than that; say 0.05. So, first let's implement the kernel.

<sup>I totally eyeballed that like I said, and definitely didn't just generate the true function in the first place from an RBF function with that bandwidth....</sup>

### Kernels

So, first, let's implement an RBF kernel first. I've put some helper infrastructure in the `LazyKernel` class (in [`ds3_support.kernels`](ds3_support/kernels.py)) that will be especially useful later; for now, it's just a way to organize computing it. Here's an example of how to use it:

In [ ]:
class LinearKernel(LazyKernel):
    def _compute(self, A, B):
        return A @ B.t()

The `_compute` method computes the kernel between two inputs `A` and `B`. (`.t()` is PyTorch for taking a transpose; `@` is the nifty Python 3.6+ syntax for matrix multiplication.)

The `LazyKernel` base class lets us use this in various ways. First, to find the kernel from one set of points to another:

In [ ]:
K = LinearKernel(toy_X_train, toy_X_test)
print(K)
K.XY

You can also get the X-to-X (`XX`) and Y-to-Y (`YY`) kernel matrices from the same object (which are the result of `_compute(X, X)` and `_compute(Y, Y)`). These aren't computed until you need them, but then they're cached after you use them the first time; this is why it's a `LazyKernel`.

In [ ]:
K.XX

If you only want the kernel matrix for a dataset to itself, you can just not pass the second argument. Then K.XY won't exist.

In [ ]:
LinearKernel(toy_X_train).XX

Alternatively, you can pass `None`, which is a special value meaning "use the first one." Then `XY` and so on will exist, but it knows to cache them appropriately.

You can also pass three arguments; then there'll be `XZ`, etc. You can also access them with e.g. `K[0, 2]`.

In [ ]:
K = LinearKernel(toy_X_train, None, toy_X_test)
print(K)
K.YZ

Here's an example of a slightly more complex kernel class, with some parameters:

In [ ]:
class PolynomialKernel(LazyKernel):
    def __init__(self, X, *rest, degree=3, gamma=None, coef0=1):
        super().__init__(X, *rest)
        self.degree = degree
        self.gamma = 1 / X.shape[1] if gamma is None else gamma
        self.coef0 = coef0

    def _compute(self, A, B):
        XY = A @ B.t()
        return (self.gamma * XY + self.coef0) ** self.degree

You also totally don't need to do this – you definitely won't notice the difference on this scale of data – but if you want to cache some computation for each dataset, you can use the `_precompute` interface. You can return a list of cached information in `_precompute`, which then get passed to `_compute` as `_compute(A, *A_precomputed, B, *B_precomputed)`.

In [ ]:
class LinearAndSquareKernel(LazyKernel):
    def _precompute(self, A):
        return [A * A]
    
    def _compute(self, A, A_squared, B, B_squared):
        return A @ B.t() + A_squared @ B_squared.t()

Remember that the Gaussian RBF kernel is
$$k(x, y) = \exp\left( -\frac{1}{2 \sigma^2} \lVert x - y \rVert^2 \right).$$
Go ahead and implement that here. It might be helpful to recall that
$$\lVert x - y \rVert^2 = \lVert x \rVert^2 + \lVert y \rVert^2 - 2 x^T y.$$

In [ ]:
class RBFKernel(LazyKernel):
    def __init__(self, *parts, sigma=1):
        super().__init__(*parts)
        self.sigma = sigma
    
    # TODO: implement _compute (maybe with _precompute)
    def _precompute(self, A):                                            # SOLUTION
        # Squared norms of each data point                               # SOLUTION
        return [torch.einsum("ij,ij->i", A, A)]                          # SOLUTION
                                                                         # SOLUTION
    def _compute(self, A, A_sqnorms, B, B_sqnorms):                      # SOLUTION
        D2 = A_sqnorms[:, None] + B_sqnorms[None, :] - 2 * (A @ B.t())   # SOLUTION
        return torch.exp(D2 / (-2 * self.sigma ** 2))                    # SOLUTION

You can check your implementation against scikit-learn's implementation (but it doesn't work in PyTorch, so don't just use it directly):

In [ ]:
sigma = np.random.lognormal()
K = RBFKernel(toy_X_train, toy_X_test, sigma=sigma)

from sklearn.metrics.pairwise import rbf_kernel
gamma = 1 / (2 * sigma**2)  # sklearn uses this parameterization
assert np.allclose(K.XX.numpy(), rbf_kernel(toy_X_train, gamma=gamma))
assert np.allclose(K.XY.numpy(), rbf_kernel(toy_X_train, toy_X_test, gamma=gamma))
assert np.allclose(K.YY.numpy(), rbf_kernel(toy_X_test, gamma=gamma))

del rbf_kernel

### Implementing ridge regression

Okay, now we can compute all kinds of kernels; let's actually use them for something. First, let's implement ridge regression. Remember that ridge regression is
$$
  \min_{f \in \mathcal{H}} \frac{1}{n} \sum_{i=1}^n \lVert f(X_i) - y_i \rVert^2 + \frac12 \lambda \lVert f \rVert_{\mathcal{H}}^2 
.$$
The representer theorem tells us that $f(x) = \sum_{i=1}^n \alpha_i k(X_i, x)$,
and then some calculus gives us
$$\alpha = (K + n \lambda I)^{-1} y, \tag{*}$$
where $K_{ij} = k(X_i, X_j)$.

There's also one more wrinkle: we're going to need to compute ridge regression for multiple outputs $y$ at once with the same $X$s. This just means solving the linear system for more than one $y$ at once; most software, including PyTorch, supports this built-in.

To implement $\textrm{(*)}$, there are (at least) two approaches.

- [`torch.solve`](https://pytorch.org/docs/stable/torch.html#torch.solve) is a general-purpose matrix solver, which uses an [LU decomposition](https://en.wikipedia.org/wiki/LU_decomposition). This doesn't exploit the special structure of kernel matrices (namely that they're [positive-definite](https://en.wikipedia.org/wiki/Definiteness_of_a_matrix)).

- You could also use a [Cholesky factorization](https://en.wikipedia.org/wiki/Cholesky_factorization), which is the standard approach for positive-definite matrices, and will be somewhat faster. You could implement this with [`torch.cholesky`](https://pytorch.org/docs/stable/torch.html#torch.cholesky) and [`torch.cholesky_solve`](https://pytorch.org/docs/stable/torch.html#torch.cholesky_solve) – but unfortunately PyTorch hasn't implemented derivatives through `cholesky_solve` yet, and we're going to need that later. You can use [`torch.triangular_solve`](https://pytorch.org/docs/stable/torch.html#torch.triangular_solve) instead; Cholesky gives us $L$ such that $K + n \lambda I = L L^T$, so you'll want $\alpha = (L L^T)^{-1} y = L^{-T} (L^{-1} y)$. (Make sure to pass `upper=False` to `triangular_solve`.)

In [ ]:
class KernelRidgeRegression:
    def __init__(
        self,
        reg_wt=1,
        solver='cholesky',  # SOLUTION
    ):
        """
        reg_wt: The regularization weight lambda.
        """
        self.reg_wt = reg_wt
        self.solver = solver  # SOLUTION
    
    def fit(self, K_XX, y):
        """
        Fit the ridge regression:
        
          K_XX: The training kernel matrix, of shape [n_train, n_train].
          y: The training labels, of shape [n_train] or [n_train, n_labels].        
        """
        K_XX, y = as_tensors(K_XX, y)
        
        assert len(K_XX.shape) == 2
        self.n_train_ = K_XX.shape[0]
        
        if len(y.shape) == 1:
            self.n_labels_ = None
            y = y[:, None]
        else:
            assert len(y.shape) == 2
            self.n_labels_ = y.shape[1]

        assert K_XX.shape[1] == y.shape[0] == self.n_train_
        
        # TODO: find the solution, and save it in self.alpha_
        # If you set anything
        to_inv = K_XX + self.reg_wt * torch.eye(K_XX.shape[0])          # SOLUTION
        if self.solver == 'lu':                                         # SOLUTION
            # the LU-based solution                                     # SOLUTION
            self.alpha_ = torch.solve(y, to_inv)[0]                     # SOLUTION
        elif self.solver == 'cholesky':                                 # SOLUTION
            # the cholesky solution (default)                           # SOLUTION
            L = torch.cholesky(to_inv, upper=False)                     # SOLUTION
            inner = torch.triangular_solve(y, L, upper=False).solution  # SOLUTION
            self.alpha_ = torch.triangular_solve(                       # SOLUTION
                inner, L, upper=False, transpose=True).solution         # SOLUTION
        else:                                                           # SOLUTION
            raise ValueError(f"unknown solver {self.solver!r}")         # SOLUTION
    
    def predict(self, K_test):
        """
        Predict the labels of a test set.
        
          K_test: The train-to-test kernel matrix, shape [n_train, n_test].
        
        Return: the vector of test predictions, shape [n_test].
        """
        K_test = torch.as_tensor(K_test)
        assert len(K_test.shape) == 2
        assert K_test.shape[0] == self.n_train_
        
        # TODO: set preds to shape [n_test, n_labels]
        preds = K_test.t() @ self.alpha_     # SOLUTION
        
        # return a vector if we were fit with a vector, matrix otherwise
        return preds.squeeze(1) if self.n_labels_ is None else preds
    
    def mses(self, K_test, y_test):
        """
        Computes the mean squared error of each output for a test set.
        
          K_test: The train-to-test kernel matrix, shape [n_train, n_test].
          y_test: The test labels, shape [n_train] or [n_train, n_labels]
                  (agreeing with what was passed to fit()).
                  
        Returns a vector if y_test is 2d, or a scalar if not.
        """
        preds = self.predict(K_test)
        y_test = torch.as_tensor(y_test)
        assert preds.shape == y_test.shape
        return ((preds - y_test) ** 2).mean(0)
    
    def mse(self, K_test, y_test):
        """
        Computes the mean squared error across outputs for a test set.
        
          K_test: The train-to-test kernel matrix, shape [n_train, n_test].
          y_test: The test labels, shape [n_train] or [n_train, n_labels]
                  (agreeing with what was passed to fit()).
                  
        Returns a scalar.
        """
        return self.mses(K_test, y_test).mean()

Does it work?

In [ ]:
def evaluate(krr, kernel_cls=None, plot=True,
             X_train=None, y_train=None, X_test=None, y_test=None):
    if X_train is None:
        X_train = toy_X_train
    if y_train is None:
        y_train = toy_y_train
    if X_test is None:
        X_test = toy_X_test
    if y_test is None:
        y_test = toy_y_test
    
    plot_xs = np.linspace(0, 1, num=500)[:, None]
    if kernel_cls is None:
        kernel_cls = functools.partial(RBFKernel, sigma=0.05)
    K = kernel_cls(X_train, X_test, plot_xs)

    krr.fit(K.XX, y_train)
    train_mse = krr.mse(K.XX, y_train).item()
    test_mse = krr.mse(K.XY, y_test).item()
    
    if plot:
        fig, ax = plt.subplots()
        ax.scatter(X_train[:, 0], y_train)
        ax.plot(plot_xs, krr.predict(K.XZ).numpy(), lw=4, color='r')
        ax.set_title(f"Train MSE: {train_mse:.2} / Test MSE: {test_mse:.2}")
    else:
        return train_mse, test_mse

In [ ]:
evaluate(KernelRidgeRegression(reg_wt=.1))
plt.savefig('figs/toy-krr-eval.png'); plt.close()  # SOLUTION

Here's what my solution looks like:
![hi](figs/toy-krr-eval.png)

In [ ]:
# SOLUTION CELL
# Check that the LU solve also works; should be almost identical.
evaluate(KernelRidgeRegression(reg_wt=.1, solver='lu'))

### Choice of $\lambda$
For this problem, we want a pretty small $\lambda$, just enough to make it work, since the amount of inherent noise in the problem is fairly small. To check this, let's just try a bunch of different $\lambda$s.

(Note that the Cholesky solution is likely to crash with small $\lambda$ before the `torch.solve` solution would, because – especially in the `float32` computation we're doing here – $K + n \lambda I$ will appear singular. My results image below used the `torch.solve` version.)

**Exercise**: What do you think will happen visually when you change $\lambda$? Try a few different numbers for `reg_wt`, including increasing it really high (like `1000`), and see if it matches your intuition.

In [ ]:
evaluate(KernelRidgeRegression(reg_wt=5))

In [ ]:
lams = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1]
train_mses = []
test_mses = []
for lam in lams:
    krr = KernelRidgeRegression(reg_wt=lam)
    krr.solver = 'lu'    # SOLUTION
    try:
        train, test = evaluate(krr, plot=False)
    except RuntimeError as e:
        print(f"{lam}: {e}")
        train = test = np.nan
    train_mses.append(train)
    test_mses.append(test)
    
fig, ax = plt.subplots()
ax.plot(lams, train_mses, marker='o', label='train MSE')
ax.plot(lams, test_mses, marker='o', label='test MSE')
ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'$\lambda$')
fig.savefig('figs/toy-krr-lambda.png'); plt.close()  # SOLUTION

My results, for reference, look like:
![](figs/toy-krr-lambda.png)

My Cholesky solution failed for $\lambda < 10^{-3}$, so your plot might cut off the first two points. We can actually tell that the behavior for these small values of $\lambda$ is numerical error, not overfitting: overfitting would increase the test error, but have the same or lower training error.

### Choosing the kernel

We can also see whether my guess at $\sigma$ was really right.

**Exercise:** First, what happens when you change $\sigma$? Try some numbers, including much larger and much smaller ones, and see if it looks like what you expect to happen. See how this interacts with setting $\lambda$ as well.

In [ ]:
evaluate(KernelRidgeRegression(reg_wt=.01),
         kernel_cls=functools.partial(RBFKernel, sigma=.005))

**Exercise:** What happens if you use `LinearKernel`? `PolynomialKernel`, with different `degree`?

In [ ]:
evaluate(KernelRidgeRegression(reg_wt=.1),
         kernel_cls=functools.partial(LinearKernel))

Going back to `RBFKernel`, let's do a joint search over $\lambda$ and $\sigma$, since they'll interact with one another:

In [ ]:
lams = [1e-3, 1e-2, 1e-1, 1e0, 1e1]
sigs = [.001, .01, .05, .1, .25, .5, 1]
mses = np.full((len(lams), len(sigs), 2), np.nan)

for lam_i, lam in enumerate(lams):
    for sig_i, sig in enumerate(sigs):
        krr = KernelRidgeRegression(reg_wt=lam)
        krr.solver = 'lu'  # SOLUTION
        try:
            mses[lam_i, sig_i, :] = evaluate(
                krr, functools.partial(RBFKernel, sigma=sig), plot=False)
        except RuntimeError as e:
            print(f"{lam} {sig}: {e}")

fig, axes = plt.subplots(ncols=2, figsize=(12, 4), constrained_layout=True)

norm = mpl.colors.LogNorm(vmin=np.nanmin(mses), vmax=np.nanmax(mses))

ax1, ax2 = axes
ax1.matshow(mses[:, :, 0], norm=norm)
best1, best2 = np.unravel_index(np.argmin(mses[:, :, 0]), mses.shape[:2])
ax1.scatter([best2], [best1], marker='o', color='c', s=100)
ax1.set_title(f"Train MSE: min {mses[best1, best2, 0]:.3}")

im = ax2.matshow(mses[:, :, 1], norm=norm)
best1, best2 = np.unravel_index(np.argmin(mses[:, :, 1]), mses.shape[:2])
ax2.scatter([best2], [best1], marker='o', color='c', s=100)
ax2.set_title(f"Test MSE: min {mses[best1, best2, 1]:.3}")
fig.colorbar(im)

for ax in axes:
    ax.grid(False)
    
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(1))
    ax.xaxis.set_ticks_position('bottom')
    ax.xaxis.set_ticklabels([''] + [f'{sig}' for sig in sigs])
    ax.set_xlabel(r'$\sigma$')
    
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(1))
    ax.yaxis.set_ticklabels([''] + [f'{lam}' for lam in lams])
    ax.set_ylabel(r'$\lambda$')
    
fig.savefig('figs/toy-krr-sig-lambda.png');  # SOLUTION
# If you get a RuntimeWarning due to nans, don't worry about it...

For reference, my result looks like
![](figs/toy-krr-sig-lambda.png)
Smaller $\sigma$ can get lower training error, but the best test error is definitely with $\sigma = 0.05$.

### Shifting `y`

What do you think will happen if we fit with `y` replaced by `y + 100`?

In [ ]:
# normal fit, for comparison
evaluate(KernelRidgeRegression(reg_wt=.1))

In [ ]:
# fit with labels shifted upwards by 100
evaluate(KernelRidgeRegression(reg_wt=.1),
         y_train=toy_y_train + 100, y_test=toy_y_test + 100)

(Some

blank

space

for

you

to

think

and

then

run

it

and

then

maybe

think

some

more

before

seeing

the

discussion.)

<span style="font-size: 2em">🤔</span>

Remember that in traditional linear regression, you fit $y = w \cdot x + b$. Here, we're using $y = w \cdot \varphi(x)$; no $+ b$. Unlike in traditional linear regression, the kernel can more-or-less account for it, but the weird thing at the end – and the overall downward trend – could be fixed by adding an offset.

The easiest way to do that is to add an additional dimension to $\varphi(x)$ that's just a constant $1$, so that the corresponding element of $w$ can be $b$. Since $k(x, y) = \varphi(x) \cdot \varphi(y)$, this means that we just add 1 to the kernel.

In [ ]:
class RBFKernelPlusOne(RBFKernel):
    def _compute(self, *args):
        return super()._compute(*args) + 1

In [ ]:
evaluate(KernelRidgeRegression(reg_wt=.1),
         kernel_cls=functools.partial(RBFKernelPlusOne, sigma=.05),
         y_train=toy_y_train + 100, y_test=toy_y_test + 100)

Much better!

One thing to note: traditional ridge regression regularizes with $\lambda \lVert w \rVert^2$, and leaves $b$ unregularized. This has nice properties like making the fit shift _exactly_ along with an offset $y$. By adding 1 to the kernel, though, ours will _slightly_ change with offsets, because we're also effectively regularizing with $\lambda b^2$.

In [ ]:
evaluate(KernelRidgeRegression(reg_wt=.1),
         kernel_cls=functools.partial(RBFKernelPlusOne, sigma=.05),
         y_train=toy_y_train + 5000, y_test=toy_y_test + 5000)

There are a few ways to resolve this:

1. Actually add an unregularized offset into the model and work it out properly.
2. Mitigate the scaling by adding a large constant, instead of 1; if you add $c$, then the regularization is effectively $\lambda {b^2}/{c^2}$.
3. Just subtract the mean of `y_train` before passing it into the model, then add it on to predictions later.

People normally do option 3 in practice.

### A harder problem

Okay, that problem was too easy. Let's do something a little more interesting.

## Advanced ridge regression: Meta-learning